In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Algoritmo ni Shor

*Tansiya ng paggamit: Tatlong segundo sa isang Eagle r3 processor (TANDAAN: Ito ay isang tantiya lamang. Maaaring mag-iba ang inyong runtime.)*

[Ang algoritmo ni Shor,](https://epubs.siam.org/doi/abs/10.1137/S0036144598347011) na binuo ni Peter Shor noong 1994, ay isang makabagong quantum algorithm para sa pag-factor ng mga integer sa polynomial time. Ang kahalagahan nito ay nakasalalay sa kakayahan nitong mag-factor ng malalaking integer nang mas mabilis nang exponential kaysa sa alinmang kilalang classical algorithm, na bumabanta sa seguridad ng malawakang ginagamit na mga cryptographic system tulad ng RSA, na umaasa sa kahirapan ng pag-factor ng malalaking numero. Sa pamamagitan ng epektibong paglutas ng problemang ito sa sapat na makapangyarihang quantum computer, ang algoritmo ni Shor ay maaaring magdulot ng rebolusyon sa mga larangan tulad ng cryptography, cybersecurity, at computational mathematics, na nagpapahayag ng transformative power ng quantum computation.

Nakatuon ang tutorial na ito sa pagpapakita ng algoritmo ni Shor sa pamamagitan ng pag-factor ng 15 sa isang quantum computer.

Una, tutukuyin natin ang order finding problem at bubuo ng mga katumbas na circuit mula sa quantum phase estimation protocol. Susunod, pagaganahin natin ang mga order finding circuit sa tunay na hardware gamit ang pinakamaikling-lalim na mga circuit na maaari nating i-transpile. Ang huling seksyon ay kukumpletuhin ang algoritmo ni Shor sa pamamagitan ng pag-uugnay ng order finding problem sa integer factorization.

Tatapusin natin ang tutorial na may pag-uusap tungkol sa iba pang mga demonstrasyon ng algoritmo ni Shor sa tunay na hardware, na nakatuon sa parehong generic na mga implementasyon at sa mga iniayon para sa pag-factor ng mga partikular na integer tulad ng 15 at 21.
Tandaan: Ang tutorial na ito ay mas nakatuon sa implementasyon at demonstrasyon ng mga circuit na may kaugnayan sa algoritmo ni Shor. Para sa isang malalim na educational resource sa materyal, mangyaring sumangguni sa kursong [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction) ni Dr. John Watrous, at mga papel sa seksyong [References](#references).
### Mga Kinakailangan
Bago magsimula sa tutorial na ito, siguraduhin na mayroon kayo ng mga sumusunod na naka-install:
- Qiskit SDK v2.0 o mas bago, na may suportang [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 o mas bago (`pip install qiskit-ibm-runtime`)
### Setup

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
from math import floor, gcd, log

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.transpiler import CouplingMap, generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Hakbang 1: I-map ang mga classical input sa isang quantum problem
### Background
Ang algoritmo ni Shor para sa integer factorization ay gumagamit ng isang intermediary problem na kilala bilang *order finding* problem. Sa seksyong ito, ipapakita natin kung paano lutasin ang order finding problem gamit ang *quantum phase estimation*.
### Problema sa phase estimation
Sa phase estimation problem, binibigyan tayo ng isang quantum state na $\ket{\psi}$ ng $n$ qubit, kasama ng isang unitary quantum circuit na gumagana sa $n$ qubit. Ipinangako sa atin na ang $\ket{\psi}$ ay isang eigenvector ng unitary matrix na $U$ na naglalarawan sa aksyon ng circuit, at ang ating layunin ay kalkulahin o tantiyahin ang eigenvalue na $\lambda = e^{2 \pi i \theta}$ kung saan tumutugma ang $\ket{\psi}$. Sa madaling salita, ang circuit ay dapat mag-output ng approximation sa numerong $\theta \in [0, 1)$ na nakakatugon sa $$U \ket{\psi}= e^{2 \pi i \theta} \ket{\psi}.$$
Ang layunin ng phase estimation circuit ay tantiyahin ang $\theta$ sa $m$ bits. Sa matematika, nais nating makahanap ng $y$ na $\theta \approx y / 2^m$, kung saan ang $y \in {0, 1, 2, \dots, 2^{m-1}}$. Ang sumusunod na larawan ay nagpapakita ng quantum circuit na tumantiya ng $y$ sa $m$ bits sa pamamagitan ng paggawa ng measurement sa $m$ qubit.
![Quantum phase estimation circuit](../learning/images/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/phase-estimation-procedure.svg)
Sa circuit sa itaas, ang itaas na $m$ qubit ay sinimulan sa estado ng $\ket{0^m}$, at ang ibabang $n$ qubit ay sinimulan sa $\ket{\psi}$, na ipinangakong isang eigenvector ng $U$. Ang unang sangkap sa phase estimation circuit ay ang mga controlled-unitary operation na responsable sa pagsasagawa ng *phase kickback* sa kanilang katumbas na control qubit. Ang mga controlled unitary na ito ay naka-exponentiate alinsunod sa posisyon ng control qubit, mula sa least significant bit hanggang sa most significant bit. Dahil ang $\ket{\psi}$ ay isang eigenvector ng $U$, ang estado ng ibabang $n$ qubit ay hindi naapektuhan ng operasyong ito, ngunit ang phase information ng eigenvalue ay kumakalat sa itaas na $m$ qubit.
Lumalabas na pagkatapos ng phase kickback operation sa pamamagitan ng mga controlled-unitary, ang lahat ng posibleng estado ng itaas na $m$ qubit ay orthonormal sa isa't isa para sa bawat eigenvector na $\ket{\psi}$ ng unitary na $U$. Samakatuwid, ang mga estadong ito ay perpektong maipag-kakaiba, at maaari nating i-rotate ang basis na kanilang binubuo pabalik sa computational basis upang gumawa ng measurement. Ang isang mathematical analysis ay nagpapakita na ang rotation matrix na ito ay tumutugma sa inverse quantum Fourier transform (QFT) sa $2^m$-dimensional Hilbert space. Ang intuition sa likod nito ay ang periodic structure ng mga modular exponentiation operator ay naka-encode sa quantum state, at ang QFT ay nag-convert ng periodicity na ito sa mga nasu-sukat na peak sa frequency domain.

Para sa mas malalim na pag-unawa kung bakit ginagamit ang QFT circuit sa algoritmo ni Shor, inirerekomenda namin sa mambabasa ang kursong [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction).
Handa na tayong gumamit ng phase estimation circuit para sa order finding.
### Problema sa order finding
Upang tukuyin ang order finding problem, magsisimula tayo sa ilang konsepto ng number theory. Una, para sa anumang ibinigay na positive integer na $N$, tukuyin ang set na $\mathbb{Z}_N$ bilang $$\mathbb{Z}_N = {0, 1, 2, \dots, N-1}.$$
Ang lahat ng arithmetic operation sa $\mathbb{Z}_N$ ay ginagawa modulo $N$. Partikular, ang lahat ng elemento na $a \in \mathbb{Z}_n$ na coprime sa $N$ ay espesyal at bumubuo ng $\mathbb{Z}^*_N$ bilang $$\mathbb{Z}^*_N = { a \in \mathbb{Z}_N : \mathrm{gcd}(a, N)=1 }.$$
Para sa isang elemento na $a \in \mathbb{Z}^*_N$, ang pinakamaliit na positive integer na $r$ na $$a^r \equiv 1 \; (\mathrm{mod} \; N)$$ ay tinutukoy bilang *order* ng $a$ modulo $N$. Tulad ng makikita natin sa ibang pagkakataon, ang paghahanap ng order ng isang $a \in \mathbb{Z}^*_N$ ay magbibigay-daan sa atin upang i-factor ang $N$.
Upang buuin ang order finding circuit mula sa phase estimation circuit, kailangan natin ng dalawang pagsasaalang-alang. Una, kailangan nating tukuyin ang unitary na $U$ na magpapahintulot sa atin na mahanap ang order na $r$, at pangalawa, kailangan nating tukuyin ang isang eigenvector na $\ket{\psi}$ ng $U$ upang ihanda ang panimulang estado ng phase estimation circuit.

Upang iugnay ang order finding problem sa phase estimation, isasaalang-alang natin ang operasyon na tinukoy sa isang system na ang mga classical state ay tumutugma sa $\mathbb{Z}_N$, kung saan tayo ay nag-multiply ng isang fixed element na $a \in \mathbb{Z}^*_N$. Partikular, tutukuyin natin ang multiplication operator na $M_a$ na $$M_a \ket{x} = \ket{ax \; (\mathrm{mod} \; N)}$$ para sa bawat $x \in \mathbb{Z}_N$. Tandaan na implisito na ang produkto modulo $N$ ay kinukuha natin sa loob ng ket sa kanang bahagi ng equation. Ang isang mathematical analysis ay nagpapakita na ang $M_a$ ay isang unitary operator. Higit pa rito, lumalabas na ang $M_a$ ay may mga pares ng eigenvector at eigenvalue na nagpapahintulot sa atin na iugnay ang order na $r$ ng $a$ sa phase estimation problem. Partikular, para sa anumang pagpili ng $j \in {0, \dots, r-1}$, mayroon tayo na $$\ket{\psi_j} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \omega^{-jk}_{r} \ket{a^k}$$ ay isang eigenvector ng $M_a$ na ang katumbas na eigenvalue ay $\omega^{j}_{r}$, kung saan $$\omega^{j}_{r} = e^{2 \pi i \frac{j}{r}}.$$
Sa pag-obserba, nakikita natin na ang isang convenient na pares ng eigenvector/eigenvalue ay ang estado na $\ket{\psi_1}$ na may $\omega^{1}_{r} = e^{2 \pi i \frac{1}{r}}$. Samakatuwid, kung makakahanap tayo ng eigenvector na $\ket{\psi_1}$, maaari nating tantiyahin ang phase na $\theta=1/r$ gamit ang ating quantum circuit at sa gayon ay makakuha ng tantiya ng order na $r$. Gayunpaman, hindi ito madali gawin, at kailangan nating isaalang-alang ang alternatibo.

Isaalang-alang natin kung ano ang magreresulta sa circuit kung ihanda natin ang computational state na $\ket{1}$ bilang panimulang estado. Ito ay hindi isang eigenstate ng $M_a$, ngunit ito ay ang uniform superposition ng mga eigenstate na aming inilarawan sa itaas. Sa madaling salita, ang sumusunod na relasyon ay totoo. $$ \ket{1} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \ket{\psi_k} $$
Ang implikasyon ng equation sa itaas ay kung itakda natin ang panimulang estado sa $\ket{1}$, makakakuha tayo ng eksaktong parehong resulta ng measurement na parang pumili tayo ng $k \in { 0, \dots, r-1}$ nang pantay-pantay na random at ginamit ang $\ket{\psi_k}$ bilang eigenvector sa phase estimation circuit. Sa madaling salita, ang isang measurement ng itaas na $m$ qubit ay nagbubunga ng approximation na $y / 2^m$ sa halaga ng $k / r$ kung saan ang $k \in { 0, \dots, r-1}$ ay pinili nang pantay-pantay na random. Ito ay nagbibigay-daan sa atin na matutunan ang $r$ na may mataas na antas ng kumpiyansa pagkatapos ng ilang independyenteng run, na siyang ating layunin.
### Mga modular exponentiation operator
Sa ngayon, inugnay natin ang phase estimation problem sa order finding problem sa pamamagitan ng pagtukoy ng $U = M_a$ at $\ket{\psi} = \ket{1}$ sa ating quantum circuit. Samakatuwid, ang huling natitirang sangkap ay ang paghahanap ng isang epektibong paraan upang tukuyin ang mga modular exponential ng $M_a$ bilang $M_a^k$ para sa $k = 1, 2, 4, \dots, 2^{m-1}$.
Upang isagawa ang computation na ito, nakikita natin na para sa anumang power na $k$ na ating pipiliin, maaari tayong lumikha ng circuit para sa $M_a^k$ hindi sa pamamagitan ng pag-uulit ng $k$ beses ang circuit para sa $M_a$, kundi sa halip sa pamamagitan ng pagkalkula ng $b = a^k \; \mathrm{mod} \; N$ at pagkatapos ay paggamit ng circuit para sa $M_b$. Dahil kailangan lang natin ng mga power na mga power din ng 2, maaari nating gawin ito nang epektibo sa classical gamit ang iterative squaring.
## Hakbang 2: I-optimize ang problema para sa quantum hardware execution
### Partikular na halimbawa na may $N = 15$ at $a=2$
Maaari tayong huminto dito upang talakayin ang isang partikular na halimbawa at buuin ang order finding circuit para sa $N=15$. Tandaan na ang posibleng nontrivial na $a \in \mathbb{Z}_N^*$ para sa $N=15$ ay $a \in {2, 4, 7, 8, 11, 13, 14 }$. Para sa halimbawang ito, pipiliin natin ang $a=2$. Bubuo tayo ng $M_2$ operator at ng mga modular exponentiation operator na $M_2^k$.
Ang aksyon ng $M_2$ sa mga computational basis state ay ang mga sumusunod.
$$M_2 \ket{0} = \ket{0} \quad M_2 \ket{5} = \ket{10} \quad M_2 \ket{10} = \ket{5}$$
$$M_2 \ket{1} = \ket{2} \quad M_2 \ket{6} = \ket{12} \quad M_2 \ket{11} = \ket{7}$$
$$M_2 \ket{2} = \ket{4} \quad M_2 \ket{7} = \ket{14} \quad M_2 \ket{12} = \ket{9}$$
$$M_2 \ket{3} = \ket{6} \quad M_2 \ket{8} = \ket{1} \quad M_2 \ket{13} = \ket{11}$$
$$M_2 \ket{4} = \ket{8} \quad M_2 \ket{9} = \ket{3} \quad M_2 \ket{14} = \ket{13}$$
Sa pag-obserba, nakikita natin na ang mga basis state ay na-shuffle, kaya mayroon tayong permutation matrix. Maaari nating buuin ang operasyong ito sa apat na qubit gamit ang mga swap gate. Sa ibaba, bubuo tayo ng $M_2$ at ng mga controlled-$M_2$ operation.

In [2]:
def M2mod15():
    """
    M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [3]:
# Get the M2 operator
M2 = M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0a8885f1-91d4-40bd-912d-dc5eea05f5bd-0.avif" alt="Output of the previous code cell" />

In [4]:
def controlled_M2mod15():
    """
    Controlled M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [5]:
# Get the controlled-M2 operator
controlled_M2 = controlled_M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M2, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif)

Ang mga gate na gumagana sa mahigit dalawang qubit ay pag-hahatiin pa sa mga two-qubit gate.

In [6]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif)

Ngayon kailangan nating buuin ang mga modular exponentiation operator. Upang makakuha ng sapat na precision sa phase estimation, gagamitin natin ang walong qubit para sa estimation measurement. Samakatuwid, kailangan nating buuin ang $M_b$ na may $b = a^{2^k} \; (\mathrm{mod} \; N)$ para sa bawat $k = 0, 1, \dots, 7$.

In [7]:
def a2kmodN(a, k, N):
    """Compute a^{2^k} (mod N) by repeated squaring"""
    for _ in range(k):
        a = int(np.mod(a**2, N))
    return a

In [8]:
k_list = range(8)
b_list = [a2kmodN(2, k, 15) for k in k_list]

print(b_list)

[2, 4, 1, 1, 1, 1, 1, 1]


As we can see from the list of $b$ values, in addition to $M_2$ that we previously constructed, we also need to build $M_4$ and $M_1$. Note that $M_1$ acts trivially on the computational basis states, so it is simply the identity operator.

$M_4$ acts on the computational basis states as follows.
$$M_4 \ket{0} = \ket{0} \quad M_4 \ket{5} = \ket{5} \quad M_4 \ket{10} = \ket{10}$$
$$M_4 \ket{1} = \ket{4} \quad M_4 \ket{6} = \ket{9} \quad M_4 \ket{11} = \ket{14}$$
$$M_4 \ket{2} = \ket{8} \quad M_4 \ket{7} = \ket{13} \quad M_4 \ket{12} = \ket{3}$$
$$M_4 \ket{3} = \ket{12} \quad M_4 \ket{8} = \ket{2} \quad M_4 \ket{13} = \ket{7}$$
$$M_4 \ket{4} = \ket{1} \quad M_4 \ket{9} = \ket{6} \quad M_4 \ket{14} = \ket{11}$$

Therefore, this permutation can be constructed with the following swap operation.

In [9]:
def M4mod15():
    """
    M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [10]:
# Get the M4 operator
M4 = M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M4, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/be041e3d-28b1-453e-983e-184c2366aeb9-0.avif" alt="Output of the previous code cell" />

In [11]:
def controlled_M4mod15():
    """
    Controlled M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [12]:
# Get the controlled-M4 operator
controlled_M4 = controlled_M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M4, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif" alt="Output of the previous code cell" />

Gates acting on more than two qubits will be further decomposed into two-qubit gates.

In [13]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/68399eef-5e55-4c95-a8a4-c8efaebd34b9-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif)

Ang mga gate na gumagana sa mahigit dalawang qubit ay pag-hahatiin pa sa mga two-qubit gate.

In [14]:
def mod_mult_gate(b, N):
    """
    Modular multiplication gate from permutation matrix.
    """
    if gcd(b, N) > 1:
        print(f"Error: gcd({b},{N}) > 1")
    else:
        n = floor(log(N - 1, 2)) + 1
        U = np.full((2**n, 2**n), 0)
        for x in range(N):
            U[b * x % N][x] = 1
        for x in range(N, 2**n):
            U[x][x] = 1
        G = UnitaryGate(U)
        G.name = f"M_{b}"
        return G

In [15]:
# Let's build M2 using the permutation matrix definition
M2_other = mod_mult_gate(2, 15)

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2_other, inplace=True)
circ = circ.decompose()

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.decompose().draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 94
2q-size: 96
Operator counts: OrderedDict({'cx': 45, 'swap': 32, 'u': 24, 'u1': 7, 'u3': 4, 'unitary': 3, 'circuit-335': 1, 'circuit-338': 1, 'circuit-341': 1, 'circuit-344': 1, 'circuit-347': 1, 'circuit-350': 1, 'circuit-353': 1, 'circuit-356': 1, 'circuit-359': 1, 'circuit-362': 1, 'circuit-365': 1, 'circuit-368': 1, 'circuit-371': 1, 'circuit-374': 1, 'circuit-377': 1, 'circuit-380': 1})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif" alt="Output of the previous code cell" />

Let's compare these counts with the compiled circuit depth of our manual implementation of the $M_2$ gate.

In [16]:
# Get the M2 operator from our manual construction
M2 = M2mod15()

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ = circ.decompose(reps=3)

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 9
2q-size: 9
Operator counts: OrderedDict({'cx': 9})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0235c931-0adb-4972-9fce-32a0341822bf-1.avif" alt="Output of the previous code cell" />

As we can see, the permutation matrix approach resulted in a significantly deep circuit even for a single $M_2$ gate compared to our manual implementation of it. Therefore, we will continue with our previous implementation of the $M_b$ operations.

Now, we are ready to construct the full order finding circuit using our previously defined controlled modular exponentiation operators. In the following code, we also import the [QFT circuit](/docs/api/qiskit/qiskit.circuit.library.QFT) from the Qiskit Circuit library, which uses Hadamard gates on each qubit, a series of controlled-U1 (or Z, depending on the phase) gates, and a layer of swap gates.

In [17]:
# Order finding problem for N = 15 with a = 2
N = 15
a = 2

# Number of qubits
num_target = floor(log(N - 1, 2)) + 1  # for modular exponentiation operators
num_control = 2 * num_target  # for enough precision of estimation

# List of M_b operators in order
k_list = range(num_control)
b_list = [a2kmodN(2, k, 15) for k in k_list]

# Initialize the circuit
control = QuantumRegister(num_control, name="C")
target = QuantumRegister(num_target, name="T")
output = ClassicalRegister(num_control, name="out")
circuit = QuantumCircuit(control, target, output)

# Initialize the target register to the state |1>
circuit.x(num_control)

# Add the Hadamard gates and controlled versions of the
# multiplication gates
for k, qubit in enumerate(control):
    circuit.h(k)
    b = b_list[k]
    if b == 2:
        circuit.compose(
            M2mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    elif b == 4:
        circuit.compose(
            M4mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    else:
        continue  # M1 is the identity operator

# Apply the inverse QFT to the control register
circuit.compose(QFT(num_control, inverse=True), qubits=control, inplace=True)

# Measure the control register
circuit.measure(control, output)

circuit.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif)

Ihambing natin ang mga bilang na ito sa compiled circuit depth ng ating manual na implementasyon ng $M_2$ gate.

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_marrakesh")
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

transpiled_circuit = pm.run(circuit)

print(
    f"2q-depth: {transpiled_circuit.depth(lambda x: x.operation.num_qubits==2)}"
)
print(
    f"2q-size: {transpiled_circuit.size(lambda x: x.operation.num_qubits==2)}"
)
print(f"Operator counts: {transpiled_circuit.count_ops()}")
transpiled_circuit.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

2q-depth: 187
2q-size: 260
Operator counts: OrderedDict({'sx': 521, 'rz': 354, 'cz': 260, 'measure': 8, 'x': 4})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif" alt="Output of the previous code cell" />

## Step 3: Execute using Qiskit primitives

First, we discuss what we would theoretically obtain if we ran this circuit on an ideal simulator. Below, we have a set of simulation results of the above circuit using 1024 shots. As we can see, we get an approximately uniform distribution over four bitstrings over the control qubits.

In [19]:
# Obtained from the simulator
counts = {"00000000": 264, "01000000": 268, "10000000": 249, "11000000": 243}

In [20]:
plot_histogram(counts)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif)

Tandaan na inalis natin ang mga controlled modular exponentiation operation mula sa natitirang control qubit dahil ang $M_1$ ay identity operator.
Tandaan na sa ibang pagkakataon sa tutorial na ito, pagaganahin natin ang circuit na ito sa `ibm_marrakish` backend. Upang gawin ito, i-transpile natin ang circuit ayon sa partikular na backend na ito at iulat ang circuit depth at mga gate count.

In [21]:
# Rows to be displayed in table
rows = []
# Corresponding phase of each bitstring
measured_phases = []

for output in counts:
    decimal = int(output, 2)  # Convert bitstring to decimal
    phase = decimal / (2**num_control)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append(
        [
            f"{output}(bin) = {decimal:>3}(dec)",
            f"{decimal}/{2 ** num_control} = {phase:.2f}",
        ]
    )

# Print the rows in a table
headers = ["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

            Register Output           Phase
0  00000000(bin) =   0(dec)    0/256 = 0.00
1  01000000(bin) =  64(dec)   64/256 = 0.25
2  10000000(bin) = 128(dec)  128/256 = 0.50
3  11000000(bin) = 192(dec)  192/256 = 0.75


Recall that the any measured phase corresponds to $\theta = k / r$ where $k$ is sampled uniformly at random from $\{0, 1, \dots, r-1 \}$. Therefore, we can use the continued fractions algorithm to attempt to find $k$ and the order $r$. Python has this functionality built in. We can use the `fractions` module to turn a float into a `Fraction` object, for example:

In [22]:
Fraction(0.666)

Fraction(5998794703657501, 9007199254740992)

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif)

## Hakbang 3: Magsagawa gamit ang mga Qiskit primitive
Una, talakayin natin kung ano ang teoretikal na makukuha natin kung pagaganahin natin ang circuit na ito sa isang ideal na simulator. Sa ibaba, mayroon tayong set ng mga resulta ng simulation ng circuit sa itaas gamit ang 1024 shot. Tulad ng makikita natin, nakakakuha tayo ng halos pantay na pamamahagi sa apat na bitstring sa mga control qubit.

In [23]:
# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)

Fraction(2, 3)

This is much nicer. The order (r) must be less than N, so we will set the maximum denominator to be `15`:

In [24]:
# Rows to be displayed in a table
rows = []

for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append(
        [phase, f"{frac.numerator}/{frac.denominator}", frac.denominator]
    )

# Print the rows in a table
headers = ["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

   Phase Fraction  Guess for r
0   0.00      0/1            1
1   0.25      1/4            4
2   0.50      1/2            2
3   0.75      3/4            4


![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif)

Sa pamamagitan ng pagsukatan sa mga control qubit, nakakakuha tayo ng walong-bit na phase estimation ng $M_a$ operator. Maaari nating i-convert ang binary representation na ito sa decimal upang mahanap ang nasukat na phase. Tulad ng makikita natin mula sa histogram sa itaas, apat na iba't ibang bitstring ang nasukat, at ang bawat isa sa kanila ay tumutugma sa isang phase value na ganito.

In [ ]:
# Sampler primitive to obtain the probability distribution
sampler = Sampler(backend)

# Turn on dynamical decoupling with sequence XpXm
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
# Enable gate twirling
sampler.options.twirling.enable_gates = True

pub = transpiled_circuit
job = sampler.run([pub], shots=1024)

In [25]:
result = job.result()[0]
counts = result.data["out"].get_counts()

In [26]:
plot_histogram(counts, figsize=(35, 5))

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/559d7030-1f67-44e8-afa7-6afc7a334677-0.avif" alt="Output of the previous code cell" />

As we can see, we obtained the same bitstrings with highest counts. Since quantum hardware has noise, there is some leakage to other bitstrings, which we can filter out statistically.

In [27]:
# Dictionary of bitstrings and their counts to keep
counts_keep = {}
# Threshold to filter
threshold = np.max(list(counts.values())) / 2

for key, value in counts.items():
    if value > threshold:
        counts_keep[key] = value

print(counts_keep)

{'00000000': 58, '01000000': 41, '11000000': 42, '10000000': 40}


Dahil nagbibigay ito ng mga fraction na ibinabalik ang resulta nang eksakto (sa kasong ito, `0.6660000...`), maaari itong magbigay ng mga komplikadong resulta tulad ng nasa itaas. Maaari nating gamitin ang `.limit_denominator()` method upang makuha ang fraction na pinaka-katulad ng ating float, na may denominator sa ibaba ng isang tiyak na halaga:

In [28]:
a = 2
N = 15

FACTOR_FOUND = False
num_attempt = 0

while not FACTOR_FOUND:
    print(f"\nATTEMPT {num_attempt}:")
    # Here, we get the bitstring by iterating over outcomes
    # of a previous hardware run with multiple shots.
    # Instead, we can also perform a single-shot measurement
    # here in the loop.
    bitstring = list(counts_keep.keys())[num_attempt]
    num_attempt += 1
    # Find the phase from measurement
    decimal = int(bitstring, 2)
    phase = decimal / (2**num_control)  # phase = k / r
    print(f"Phase: theta = {phase}")

    # Guess the order from phase
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator  # order = r
    print(f"Order of {a} modulo {N} estimated as: r = {r}")

    if phase != 0:
        # Guesses for factors are gcd(a^{r / 2} ± 1, 15)
        if r % 2 == 0:
            x = pow(a, r // 2, N) - 1
            d = gcd(x, N)
            if d > 1:
                FACTOR_FOUND = True
                print(f"*** Non-trivial factor found: {x} ***")


ATTEMPT 0:
Phase: theta = 0.0
Order of 2 modulo 15 estimated as: r = 1

ATTEMPT 1:
Phase: theta = 0.25
Order of 2 modulo 15 estimated as: r = 4
*** Non-trivial factor found: 3 ***


## Discussion

### Related work
In this section, we discuss other milestone work that has demonstrated Shor's algorithm on real hardware.

The seminal work [[3]](#references) from IBM&reg; demonstrated Shor's algorithm for the first time, factoring the number 15 into its prime factors 3 and 5 using a seven-qubit nuclear magnetic resonance (NMR) quantum computer. Another experiment [[4]](#references) factored 15 using photonic qubits. By employing a single qubit recycled multiple times and encoding the work register in higher-dimensional states, the researchers reduced the required number of qubits to one-third of that in the standard protocol, utilizing a two-photon compiled algorithm. A significant paper in the demonstration of Shor's algorithm is [[5]](#references), which uses Kitaev's iterative phase estimation [[8]](#references) technique to reduce the qubit requirement of the algorithm. Authors used seven control qubits and four cache qubits, together with the implementation of modular multipliers. This implementation, however, requires mid-circuit measurements with feed-forward operations and qubit recycling with reset operations. This demonstration was done on an ion-trap quantum computer.

More recent work [[6]](#references) focused on factoring 15, 21, and 35 on IBM Quantum&reg; hardware. Similar to previous work, researchers used a compiled version of the algorithm that employed a semi-classical quantum Fourier transform as proposed by Kitaev to minimize the number of physical qubits and gates. A most recent work [[7]](#references) also performed a proof-of-concept demonstration for factoring the integer 21. This demonstration also involved the use of a compiled version of the quantum phase estimation routine, and built upon the previous demonstration by [[4]](#references). Authors went beyond this work by using a configuration of approximate Toffoli gates with residual phase shifts. The algorithm was implemented on IBM quantum processors using only five qubits, and the presence of entanglement between the control and register qubits was verified successfully.

### Scaling of the algorithm

We note that RSA encryption typically involves key sizes on the order of 2048 to 4096 bits. Attempting to factor a 2048-bit number with Shor's algorithm will result in a quantum circuit with millions of qubits, including the error correction overhead and a circuit depth on the order of a billion, which is beyond the limits of current quantum hardware to execute. Therefore, Shor's algorithm will require either optimized circuit construction methods or robust quantum error correction to be practically viable for breaking modern cryptographic systems. We refer you to [[9]](#references) for a more detailed discussion on resource estimation for Shor's algorithm.

## Challenge

Congratulations for finishing the tutorial! Now is a great time to test your understanding. Could you try to construct the circuit for factoring 21? You can select an $a$ of your own choice. You will need to decide on the bit accuracy of the algorithm to choose the number of qubits, and you will need to design the modular exponentiation operators $M_a$. We encourage you to try this out yourself, and then read about the methodologies shown in Fig. 9 of [[6]](#references) and Fig. 2 of [[7]](#references).

In [ ]:
def M_a_mod21():
    """
    M_a (mod 21)
    """

    # Your code here
    pass

Ito ay mas maganda. Ang order (r) ay dapat na mas mababa sa N, kaya itakda natin ang maximum denominator na `15`: